In [1]:
import os
import os.path
import numpy as np
import h5py
import json
import torch
import imageio                           # is used as like scipy 'imread'
import scipy
from scipy.misc import imread,imresize
from PIL import Image
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import string
import numpy

In [2]:
# initialization for the dataset
dataset='flickr8k'
json_path='flickr8k_raw.json'
image_folder='/home/hemanth/Documents/dataset/flickr8k/Flickr8k_Dataset/'
captions_per_image=5
threshold_freq=5
output_folder=''

In [3]:
with open(json_path, 'r') as j:
    imgs = json.load(j)

In [4]:
def prepro_captions(imgs):
    # preprocess all the caption
    print('example processed tokens:')
    for i,img in enumerate(imgs):
        img['processed_tokens'] = []
        for j,s in enumerate(img['captions']):
            if i < 5 and j == 0: print('before : ',str(s))
            txt = str(s).lower().translate(string.punctuation).strip("").split()    #translate(None, string.punctuation)
            img['processed_tokens'].append(txt)
            if i < 5 and j == 0: print('after : ',txt,'\n')
    print("\n\n")
# imgs ={ {filepath:, id: ,captions:{},processed_tokens:[]}, {filepath:, id: ,captions:{},processed_tokens:[]}, ....}
    print(imgs[0],'\n')

# tokenization and preprocessing 
prepro_captions(imgs)

example processed tokens:
before :  A child in a pink dress is climbing up a set of stairs in an entry way .
after :  ['a', 'child', 'in', 'a', 'pink', 'dress', 'is', 'climbing', 'up', 'a', 'set', 'of', 'stairs', 'in', 'an', 'entry', 'way', '.'] 

before :  A black dog and a spotted dog are fighting
after :  ['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting'] 

before :  A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .
after :  ['a', 'little', 'girl', 'covered', 'in', 'paint', 'sits', 'in', 'front', 'of', 'a', 'painted', 'rainbow', 'with', 'her', 'hands', 'in', 'a', 'bowl', '.'] 

before :  A man lays on a bench while his dog sits by him .
after :  ['a', 'man', 'lays', 'on', 'a', 'bench', 'while', 'his', 'dog', 'sits', 'by', 'him', '.'] 

before :  A man in an orange hat starring at something .
after :  ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.'] 




{'file_path': '/home/hemanth/Documents/

In [5]:
t=0
v=0
for i,img in enumerate(imgs):
    if img['file_path'][58] == 't':
        t=t+1
    else:
        v=v+1
print("train : %d"%t, "val : %d"%v)
print("\nActual Total no of captions:",5* (t+v))

train : 6000 val : 1000

Actual Total no of captions: 35000


In [6]:
# max length?
# word frequency?
sent_lengths = {}
word_freq=Counter()
totcap=0
for img in imgs:
    for txt in img['processed_tokens']:
        totcap=totcap+1
        word_freq.update(txt)
        nw = len(txt)
        sent_lengths[nw] = sent_lengths.get(nw, 0) + 1

max_len = max(sent_lengths.keys())
print('Calculated & processed total no of captions :  %d' % totcap)
print('max length sentence in raw data :  %d\n' % max_len)
print('sentence length distribution (count, number of words):')
sum_len = sum(sent_lengths.values())
for i in range(max_len+1):
    print('%2d: %10d   %f%%' % (i, sent_lengths.get(i,0), sent_lengths.get(i,0)*100.0/sum_len))
print("\nWord Frequencies were counted successfully..!!")
#print('\nWord frequencies:\n\n',word_freq)

Calculated & processed total no of captions :  35000
max length sentence in raw data :  38

sentence length distribution (count, number of words):
 0:          0   0.000000%
 1:          0   0.000000%
 2:          9   0.025714%
 3:         33   0.094286%
 4:        199   0.568571%
 5:        471   1.345714%
 6:       1127   3.220000%
 7:       2328   6.651429%
 8:       2996   8.560000%
 9:       3340   9.542857%
10:       3758   10.737143%
11:       3766   10.760000%
12:       3534   10.097143%
13:       3144   8.982857%
14:       2664   7.611429%
15:       2130   6.085714%
16:       1553   4.437143%
17:       1141   3.260000%
18:        823   2.351429%
19:        636   1.817143%
20:        434   1.240000%
21:        288   0.822857%
22:        163   0.465714%
23:        161   0.460000%
24:         99   0.282857%
25:         64   0.182857%
26:         53   0.151429%
27:         26   0.074286%
28:         22   0.062857%
29:          8   0.022857%
30:          9   0.025714%
31:          

In [7]:
# thresholding the words
# creating word map or vocab mapping  'word to index' and 'index to word'
vocab=[w for w in list(word_freq.keys()) if word_freq[w] > threshold_freq]

w2i = {k: v + 1 for v, k in enumerate(vocab)}
w2i['<unk>'] = len(w2i) + 1
w2i['<start>'] = len(w2i) + 1
w2i['<end>'] = len(w2i) + 1
w2i['<pad>'] = 0

i2w = {v + 1: k for v, k in enumerate(vocab)}
i2w[len(i2w) + 1] = '<unk>'
i2w[len(i2w) + 1] = '<start>'
i2w[len(i2w) + 1] = '<end>'
i2w[0]='<pad>'
# saving the 'w2i and i2w' dictionary by json file format
with open('w2i_coco.json','w') as j:
    json.dump(w2i, j)
with open('i2w_coco.json','w') as j:
    json.dump(i2w,j)
print('"Word - Index" mapping is done!!..')

"Word - Index" mapping is done!!..


In [8]:
imgs[0]['file_path']

'/home/hemanth/Documents/dataset/flickr8k/Flickr8k_Dataset/train/1000268201_693b08cb0e.jpg'

In [9]:
print("train and val dataset splitting...")
# spliting the train and val datasets seperately..  
train_image_paths=[]
train_image_captions=[]
val_image_paths=[]
val_image_captions=[]
for img in tqdm(imgs,position=0,leave=True):
    impath = img['file_path']
    imcaps = img['processed_tokens']
    if img['file_path'][58] == 't':
        train_image_paths.append(impath)
        train_image_captions.append(imcaps)
    elif img['file_path'][58] == 'v':
        val_image_paths.append(impath)
        val_image_captions.append(imcaps)
    else:
        print('ERROR............in assigining dataset paths...........!!!!!!!!!')

100%|██████████| 7000/7000 [00:00<00:00, 1311363.97it/s]

train and val dataset splitting...


In [24]:
base_filename = dataset + '_' + str(captions_per_image) + '_cap_per_img_' + str(threshold_freq) + '_min_word_freq'
seed(123)
for impaths, imcaps, split in [(train_image_paths, train_image_captions, 'TRAIN'),
                                   (val_image_paths, val_image_captions, 'VAL')]:

    # removing the same file if already existing..
    rem_file=os.path.join(output_folder, split + '_IMAGES_' + base_filename + '.hdf5')
    if os.path.exists(rem_file):
        os.remove(rem_file)
    # saving in hdf5 file
    print("HDF5 file creation for %s started...!!!"%split)
    with h5py.File(os.path.join(output_folder, split + '_IMAGES_' + base_filename + '.hdf5'), 'a') as h:
        h.attrs['captions_per_image'] = captions_per_image

        # Create dataset inside HDF5 file to store images
        images = h.create_dataset('images', (len(impaths), 3, 256, 256), dtype='uint8')

        print("\nReading %s images and captions, storing to file...\n" % split)

        enc_captions = []
        caplens = []

        for i, path in enumerate(tqdm(impaths,position=0,leave=True)):

            # Sample captions
            if len(imcaps[i]) < captions_per_image:
                captions = imcaps[i] + [choice(imcaps[i]) for _ in range(captions_per_image - len(imcaps[i]))]
            else:
                captions = sample(imcaps[i], k=captions_per_image)

            # Sanity check
            assert len(captions) == captions_per_image

            # Read images
            #from skimage import io         im = io.imread(self.file_image)
            img = imread(impaths[i])
            if len(img.shape) == 2:
                img = img[:, :, np.newaxis]
                img = np.concatenate([img, img, img], axis=2)
            img = imresize(img, (256, 256))
            img = img.transpose(2, 0, 1)
            assert img.shape == (3, 256, 256)
            assert np.max(img) <= 255

            # Save image to HDF5 file
            images[i] = img

            for j, c in enumerate(captions):
                # Encode captions
                enc_c = [w2i['<start>']] + [w2i.get(word, w2i['<unk>']) for word in c] + [
                        w2i['<end>']] + [w2i['<pad>']] * (max_len - len(c))

                # Find caption lengths
                c_len = len(c) + 2

                enc_captions.append(enc_c)
                caplens.append(c_len)

        print("\nHDF5 file for %s created successfully.......!!!!\n"%split)
        # Sanity check
        assert images.shape[0] * captions_per_image == len(enc_captions) == len(caplens)
        # Save encoded captions and their lengths to JSON files
        with open(os.path.join(output_folder, split + '_CAPTIONS_' + base_filename + '.json'), 'w') as j:
                json.dump(enc_captions, j)

        with open(os.path.join(output_folder, split + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
                json.dump(caplens, j)
        print("JSON file for %s is created"%split)

  0%|          | 0/6000 [00:00<?, ?it/s]/home/hemanth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/hemanth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  0%|          | 13/6000 [00:00<00:48, 123.93it/s]

HDF5 file creation for TRAIN started...!!!

Reading TRAIN images and captions, storing to file...



100%|██████████| 6000/6000 [00:47<00:00, 126.14it/s]



HDF5 file for TRAIN created successfully.......!!!!



  1%|▏         | 13/1000 [00:00<00:08, 118.95it/s]

JSON file for TRAIN is created
HDF5 file creation for VAL started...!!!

Reading VAL images and captions, storing to file...



100%|██████████| 1000/1000 [00:08<00:00, 120.04it/s]



HDF5 file for VAL created successfully.......!!!!

JSON file for VAL is created


In [25]:
print("\nJSON file created successfully.......!!!!\n")
print("Input files were preprocessed successfully...........!!!!!")


JSON file created successfully.......!!!!

Input files were preprocessed successfully...........!!!!!
